In [291]:
from __future__ import  print_function
import numpy as np
import pandas as pd#origin

In [292]:
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')
n_users = users.shape[0]
print('Number of users:', n_users)

Number of users: 943


In [293]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols)
ratings_base.head()#origin

,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [294]:
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols)
ratings_test.head()#origin

,user_id,movie_id,rating,unix_timestamp
0,1,20,4,887431883
1,1,33,4,878542699
2,1,61,4,878542420
3,1,117,3,874965739
4,1,155,2,878542201


In [295]:
rate_train = ratings_base.to_numpy()
rate_train

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]], shape=(90570, 4))

In [296]:
rate_test = ratings_test.to_numpy()
rate_test

array([[        1,        20,         4, 887431883],
       [        1,        33,         4, 878542699],
       [        1,        61,         4, 878542420],
       ...,
       [      943,       570,         1, 888640125],
       [      943,       808,         4, 888639868],
       [      943,      1067,         2, 875501756]], shape=(9430, 4))

In [297]:
print('Number of traing rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])

Number of traing rates: 90570
Number of test rates: 9430


In [298]:
#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols , encoding='latin-1')

n_items = items.shape[0]
print('Number of items:', n_items)

Number of items: 1682


In [299]:
print(f'Number of items: {items.shape[0]}')
print(f'Number of attribute: {items.shape[1]}') #origin

Number of items: 1682
Number of attribute: 24


Xây dựng đặc trưng cho mỗi phim chỉ dựa vào thể loại sử dụng TF-IDF

In [300]:
X0 = items.to_numpy()
X_train_counts = X0[:, -19:]

In [301]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf= True, norm= 'l2')
#IDF = log(n_samples / df) -> smooth_idf = True => IDF = log(n_samples / (df + 1))
X = transformer.fit_transform(X= X_train_counts.tolist()).toarray()
X #origin

array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.54, 0.65, ..., 0.54, 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 1.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]], shape=(1682, 19))

Mỗi dòng của ma trận rate_train cho biết người nào đánh gia phim nào bao nhiêu điểm 

VD: dòng 1 cho biết người 1 đánh giá phim 1 là 5 điểm vào thời gian được cho ở cột cuối

In [302]:
print(rate_train[:4,:]) #origin

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]]


Xây dựng hàm tìm những phim mà người dùng đã đánh giá

In [303]:
def get_items_rated_by_user(rate_matrix, user_id): #origin
    y = rate_matrix[:,0]                        # Lấy tất cả người dùng
    ids = np.where(y == user_id + 1)[0]         # Chọn ra người dùng có tên là user_id
    items_ids = rate_matrix[ids,1] - 1          # Chọn ra những phim mà người đó đã đánh giá
    scores = rate_matrix[ids,2]                 # Chọn ra điểm của những phim đó
    return (items_ids, scores)

Học mô hình Rigde Regression cho mỗi user

In [304]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
#origin
d = X.shape[1]
W = np.zeros((d, n_users))
b = np.zeros(n_users)

for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_matrix= rate_train, user_id= n)
    Xhat = X[ids,:]
    model = Ridge(alpha= 0.01, fit_intercept= True)
    model.fit(X= Xhat, y= scores)
    W[:,n] = model.coef_
    b[n] = model.intercept_


In [305]:
Yhat = X.dot(W) + b #origin
Yhat.shape

(1682, 943)

In [306]:
n = 10 #origin
np.set_printoptions(precision= 2)
ids, scores = get_items_rated_by_user(rate_matrix= rate_test, user_id= n)
print(f"Rated movie ids: {ids}")
print(f"Predicted ratings: {Yhat[ids, n]}")
print(f"True ratings: {scores}")

Rated movie ids: [ 37 109 110 226 424 557 722 724 731 739]
Predicted ratings: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]
True ratings: [3 3 4 3 4 3 5 3 3 4]


In [307]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return np.sqrt(se/cnt)

print('RMSE for training: %.2f' %evaluate(Yhat, rate_train, W, b))
print('RMSE for test    : %.2f' %evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.91
RMSE for test    : 1.27


In [314]:
y = [1,2,4,5,6,7,5,6,5,4,2,5]
y = np.array(y)
ids = np.where(y == 5)[0]
ids

array([ 3,  6,  8, 11])